# 🗫 Machine Translation

# Setup and Data Preprocessing 



In [1]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!pip install torch==2.3.0 torchtext==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 92.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━

In [2]:
import random
import math
import copy
import io
import os

import torch
import torch.nn as nn
import numpy as np
from collections import Counter
from typing import Callable, Union

import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.utils import download_from_url, extract_archive
from torchtext import vocab as torchtext_vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from timeit import default_timer as timer

In [3]:
def seed_everything(s: int):
  """
  This function allows us to set the seed for all of our random functions
  so that we can get reproducible results.

  Parameters
  ----------
  s : int
      seed to seed all random functions with
  """
  random.seed(s)
  torch.manual_seed(s)
  torch.cuda.manual_seed_all(s)
  np.random.seed(s)
  torch.backends.cudnn.deterministic = True

SEED = 42
seed_everything(SEED)

In [4]:
# data loaded from Multi30k dataset, see https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html
url_base = "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/"
train_urls = ("train.de.gz", "train.en.gz")
val_urls = ("val.de.gz", "val.en.gz")
test_urls = ("test_2016_flickr.de.gz", "test_2016_flickr.en.gz")

# download and extract data
train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

# get tokenizers for English and German
de_tokenizer = get_tokenizer("spacy", language="de_core_news_sm")
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

100%|██████████| 637k/637k [00:00<00:00, 63.4MB/s]
100%|██████████| 569k/569k [00:00<00:00, 79.6MB/s]
100%|██████████| 24.7k/24.7k [00:00<00:00, 36.4MB/s]
100%|██████████| 21.6k/21.6k [00:00<00:00, 32.9MB/s]
100%|██████████| 22.9k/22.9k [00:00<00:00, 34.9MB/s]
100%|██████████| 21.1k/21.1k [00:00<00:00, 22.6MB/s]


In [5]:
def build_vocab(filepath: str, tokenizer: Callable) -> torchtext_vocab.vocab:
    """
    Builds a vocab based on a given file of sentence and a given tokenizer

    Parameters
    ----------
    filepath : str
        filepath from which to get sentences to generate a vocabulary
    tokenizer : Callable
        function with which to tokenize the sentences

    Returns
    ----------
    torchtext.vocab.Vocab
        a PyTorch vocab object
    """
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f: # each string here is a German or English sentence
            counter.update(tokenizer(string_))
    return torchtext.vocab.vocab(counter, specials=["<unk>", "<pad>", "<bos>", "<eos>"])

# build vocabularies for both languages based on the training data
de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

# sets the default behavior for OOV words to be <unk>
de_vocab.set_default_index(0)
en_vocab.set_default_index(0)

In [6]:
def data_process(filepaths: list[str]) -> list[tuple[torch.Tensor, torch.Tensor]]:
  """
  Builds a dataset of translated sentences from a list of filepaths

  Parameters
  ----------
  filepaths : list[str]
      a list containing the filepath to a German dataset and a filepath to
      the corresponding English dataset

  Returns
  ----------
  list[tuple[torch.Tensor, torch.Tensor]]
      a list of tuples of tensors, where each tuple is a pair of translations
      (German, English) in the form of tensors where tokenized words are encoded
      as vocabulary indices
  """
  raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    # raw_de and raw_en are paired sentences in English and German
    de_tensor_ = torch.tensor(de_vocab(de_tokenizer(raw_de)), dtype=torch.long)
    en_tensor_ = torch.tensor(en_vocab(en_tokenizer(raw_en)), dtype=torch.long)

    data.append((de_tensor_, en_tensor_))
  return data

# clean the data by tokenizing sentences and converting them to tensors
train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

In [7]:
# sets default values for the rest of the notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # remember this for later!
BATCH_SIZE = 32
PAD_IDX = de_vocab["<pad>"]
BOS_IDX = de_vocab["<bos>"]
EOS_IDX = de_vocab["<eos>"]

In [8]:
def generate_batch(data_batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
  """
  Used as the collate_fn argument when constructing DataLoaders to collate
  lists of samples into batches

  Parameters
  ----------
  data_batch : list[tuple[torch.Tensor, torch.Tensor]]
      a list of tuples of tensors, where each tuple is a pair of translations
      (German, English) in the form of tensors where tokenized words are encoded
      as vocabulary indices

  Returns
  ----------
  tuple[torch.Tensor, torch.Tensor]
      a tuple of batched German sentences and batched English sentences in the
      form of tensors of vocabulary indices
  """
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

# create data loaders for the training, validation, and test data
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=generate_batch)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=False, collate_fn=generate_batch)

# Part 1: Building a Transformer


In [30]:
class SublayerConnection(nn.Module):
    '''Applies a residual connection followed by a layer norm to any sublayer'''
    def __init__(self, size: int, dropout: float):
        """
        Initializes a SublayerConnection module

        Parameters
        ----------
        size : int
            size of the expected input to the module
        dropout : float
            dropout value to be used after the sublayer
        """
        super(SublayerConnection, self).__init__()
        self.norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, sublayer: Union[nn.Module, Callable]) -> torch.Tensor:
        """
        Forward pass of the model. Normalizes the input, applies the sublayer,
        performs a dropout, and then performs a residual connection.

        Parameters
        ----------
        x : torch.Tensor
            input Tensor
        sublayer : nn.Module or Callable
            layer (or function that acts as a layer) that a residual connection
            is performed over

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        return x + self.dropout(sublayer(self.norm(x)))


class PositionwiseFeedForward(nn.Module):
    ''' Implements the two-layer feedforward neural network used in the transformer.'''
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        """
        Initializes a PositionwiseFeedForward module

        Parameters
        ----------
        d_model : int
            size of the input into and output out of the feedforward layer
        d_ff : int
            hidden size of the feedforward layer
        dropout : float
            dropout value used at end of feedforward layer
        """
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the model. Normalizes the input, applies the sublayer,
        performs a dropout, and then performs a residual connection.

        Parameters
        ----------
        x : torch.Tensor
            input Tensor
        sublayer : nn.Module
            layer that a residual connection is performed over

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        return self.w_2(self.dropout(self.w_1(x).relu()))

def clones(module: nn.Module, N: int) -> nn.ModuleList:
    """
    Creates a nn.ModuleList of N identical copies of the inputted module

    Parameters
    ----------
    module : nn.Module
        module to clone
    N : int
        number of layers to clone

    Returns
    nn.ModuleList
        iterable list of the cloned layers
    ----------

    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

## Building an Encoder Block



In [31]:
class ManualEncoderLayer(nn.Module):
    def __init__(self,
                 size: int,
                 dropout: float,
                 nhead: int,
                 dim_ff: int):
        """
        Initializes a ManualEncoderLayer module

        Parameters
        ----------
        size : int
            size of the input into and output out of the feedforward layer
        dropout : float
            dropout value used for attention, residual connection, and
            feedforward layers
        nhead : int
            number of attention heads used in multi-head attention
        dim_ff : int
            hidden dimension of the feedforward layer
        """
        super(ManualEncoderLayer, self).__init__()
        # TODO: Initialize the necessary pieces of the encoder block
        # Multi-head attention layer
        self.self_attn = nn.MultiheadAttention(embed_dim=size, num_heads=nhead, dropout=dropout)

        # Feed-forward layer
        self.feed_forward = PositionwiseFeedForward(d_model=size, d_ff=dim_ff, dropout=dropout)

        # Sublayer connections
        self.sublayer_attn = SublayerConnection(size, dropout)
        self.sublayer_ff = SublayerConnection(size, dropout)

        # Size of the layer (used for layer normalization)
        self.size = size

    def forward(self, x: torch.Tensor, padding_mask: torch.Tensor):
        """
        Forward encoding pass for one layer.

        Parameters
        ----------
        x : torch.Tensor
            input Tensor
        padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the input

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        # TODO: Implement the forward pass
        x = self.sublayer_attn(x, lambda x: self.self_attn(x, x, x, key_padding_mask=padding_mask)[0])

        # Feed-forward sublayer
        x = self.sublayer_ff(x, self.feed_forward)

        return x


### Implementing the Encoder: `ManualEncoder`


In [32]:
class ManualEncoder(nn.Module):
    def __init__(self, layer: nn.Module, N: int):
        """
        Initializes a ManualEncoder module

        Parameters
        ----------
        layer : nn.Module
            a ManualEncoderLayer
        N : int
            number of encoder layers in the encoder
        """
        super(ManualEncoder, self).__init__()
        # TODO: Initialize the necessary pieces of the encoder
        # HINT: this mostly consists of making copies of your encoder layers
        self.layers = clones(layer, N)

        # Final layer normalization
        self.norm = nn.LayerNorm(layer.size)

    def forward(self, x: torch.Tensor, padding_mask: torch.Tensor):
        """
        Full encoder forward pass.

        Parameters
        ----------
        x : torch.Tensor
            input Tensor
        padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the input

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        # TODO: Implement the forward pass
        for layer in self.layers:
            x = layer(x, padding_mask)

        # Apply final layer normalization
        normalized_x = self.norm(x)

        return normalized_x

## Building a Decoder Block


### Implementing a Decoder Layer: `ManualDecoderLayer`



In [33]:
class ManualDecoderLayer(nn.Module):
    def __init__(self,
                 size: int,
                 dropout: float,
                 nhead: int,
                 dim_ff: int):
        """
        Initializes a ManualDecoderLayer module

        Parameters
        ----------
        size : int
            size of the input into and output out of the feedforward layer
        dropout : float
            dropout value used for attention, residual connection, and
            feedforward layers
        nhead : int
            number of attention heads used in multi-head attention
        dim_ff : int
            hidden dimension of the feedforward layer
        """
        super(ManualDecoderLayer, self).__init__()
        self.attn_weights = None
        # TODO: Initialize the necessary pieces of the decoder block
        # Self-attention layer for target sequence
        self.self_attn = nn.MultiheadAttention(embed_dim=size, num_heads=nhead, dropout=dropout)

        # Cross-attention layer for encoder-decoder attention
        self.cross_attn = nn.MultiheadAttention(embed_dim=size, num_heads=nhead, dropout=dropout)

        # Feed-forward layer
        self.feed_forward = PositionwiseFeedForward(d_model=size, d_ff=dim_ff, dropout=dropout)

        # Sublayer connections
        self.sublayer_self_attn = SublayerConnection(size, dropout)
        self.sublayer_cross_attn = SublayerConnection(size, dropout)
        self.sublayer_ff = SublayerConnection(size, dropout)

        # Size of the layer (used for layer normalization)
        self.size = size


    def forward(self,
                x: torch.Tensor,
                src_encoding: torch.Tensor,
                src_padding_mask: torch.Tensor,
                tgt_mask: torch.Tensor,
                tgt_padding_mask: torch.Tensor):
        """
        Forward decoding pass for one layer.

        Parameters
        ----------
        x : torch.Tensor
            input Tensor (model's target sequence)
        src_encoding : torch.Tensor
            output of the encoder
        src_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the "encoded input" (src_encoding)
        tgt_mask : torch.Tensor
            boolean mask Tensor displaying locations of tokens in target to
            mask attention to
        tgt_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the input (target sequence)

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        # TODO: Implement the forward pass
        x = self.sublayer_self_attn(
            x,
            lambda x: self.self_attn(
                x, x, x,
                attn_mask=tgt_mask,
                key_padding_mask=tgt_padding_mask
            )[0]
        )

        # Cross-attention sublayer
        x = self.sublayer_cross_attn(
            x,
            lambda x: self.cross_attn(
                x, src_encoding, src_encoding,
                key_padding_mask=src_padding_mask,
                need_weights=True
            )[0]
        )

        # Save attention weights for analysis
        _, self.attn_weights = self.cross_attn(
            x, src_encoding, src_encoding,
            key_padding_mask=src_padding_mask,
            need_weights=True
        )


        return self.sublayer_ff(x, self.feed_forward)

### Implementing the Decoder: `ManualDecoder`


In [34]:
class ManualDecoder(nn.Module):
    def __init__(self, layer: nn.Module, N: int):
        """
        Initializes a ManualDecoder module

        Parameters
        ----------
        layer : nn.Module
            a ManualDecoderLayer
        N : int
            number of decoder layers in the decoder
        """
        super(ManualDecoder, self).__init__()
        # TODO: Initialize the necessary pieces of the decoder
        # HINT: this mostly consists of making copies of your decoder layers
        self.layers = clones(layer, N)

        # Final layer normalization
        self.norm = nn.LayerNorm(layer.size)

    def forward(self,
                x: torch.Tensor,
                src_encoding: torch.Tensor,
                src_padding_mask: torch.Tensor,
                tgt_mask: torch.Tensor,
                tgt_padding_mask: torch.Tensor):
        """
        Full decoder forward pass.

        Parameters
        ----------
        x : torch.Tensor
            input Tensor (model's target sequence)
        src_encoding : torch.Tensor
            output of the encoder
        src_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the "encoded input" (src_encoding)
        tgt_mask : torch.Tensor
            boolean mask Tensor displaying locations of tokens in target to
            mask attention to
        tgt_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the input (target sequence)

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        # TODO: Implement the forward pass
        for layer in self.layers:
            x = layer(
                x, src_encoding,
                src_padding_mask=src_padding_mask,
                tgt_mask=tgt_mask,
                tgt_padding_mask=tgt_padding_mask
            )

        # Apply final layer normalization
        return self.norm(x)

## Putting Everything Together



In [35]:
## DO NOT EDIT THIS CELL
class TokenEmbedding(nn.Module):
    ''' Embedding layer with weights defined by a vocabulary and an embedding size'''
    def __init__(self, vocab_size: int, emb_size: int):
        """
        Initializes a TokenEmbedding module

        Parameters
        ----------
        vocab_size : int
            size of the vocab that embeddings will be from
        emb_size : int
            embedding size
        """
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        """
        Returns the embedding of the tokens multiplied by sqrt(emb_size)

        Parameters
        ----------
        tokens : torch.Tensor
            input Tensor in the form of batched tokenized sentences using
            vocabulary indices

        Returns
        ----------
        torch.Tensor
            output of the embeddings
        """
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class PositionalEncoding(nn.Module):
    ''' Positional encoding layer for the transformer. '''
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        """
        Initializes a PositionalEncoding module. The math here isn't important
        to have a firm grasp of, but the transformer explanation
        linked above does explain how this works visually. :)

        Parameters
        ----------
        emb_size : int
            embedding size from TokenEmbedding
        dropout : float
            dropout to be used after encoding all the inputs
        maxlen : int
            maximum length of the sentence
        """
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer("pos_embedding", pos_embedding)

    def forward(self, token_embedding: Union[int, torch.Tensor]):
        """
        Returns the sum of the token embedding and the positional embedding.
        Again, see the transformer visualization for more information.

        Parameters
        ----------
        token_embedding : torch.Tensor
            input Tensor from a TokenEmbedding layer

        Returns
        ----------
        torch.Tensor
            a positionally-encoded sentence representation
        """
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [36]:
class ManualTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_ff: int = 512,
                 dropout: float = 0.1):
        """
        Initializes a ManualTransformer module.

        Parameters
        ----------
        num_encoder_layers : int
            number of encoder layers in the transformer
        num_decoder_layers : int
            number of decoder layers in the transformer
        emb_size : int
            embedding size used for token embedding, positional encoding,
            and encoding and decoding
        nhead : int
            number of heads to use for multi-head attention in the encoder and
            decoder
        src_vocab_size : int
            vocab size of the input sentences of the transformer
        tgt_vocab_size : int
            vocab size of the target sentences of the transformer
        dim_ff : int
            hidden dimension of the feedforward layers
        dropout : float
            dropout to be used after encoding all the inputs
        """
        super(ManualTransformer, self).__init__()

        # TODO: Initialize the necessary pieces of the transformer
        # Token embeddings and positional encodings
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout)

        # Encoder and decoder
        encoder_layer = ManualEncoderLayer(size=emb_size, dropout=dropout, nhead=nhead, dim_ff=dim_ff)
        self.encoder = ManualEncoder(encoder_layer, num_encoder_layers)

        decoder_layer = ManualDecoderLayer(size=emb_size, dropout=dropout, nhead=nhead, dim_ff=dim_ff)
        self.decoder = ManualDecoder(decoder_layer, num_decoder_layers)

        # Final projection layer
        self.out = nn.Linear(emb_size, tgt_vocab_size)

        ### DO NOT EDIT ###
        # Initialize parameters with Glorot / fan_avg.
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def get_tgt_mask(self, tgt: torch.Tensor) -> torch.Tensor:
        """
        Returns a mask of the target sequence to be used with attention
        during decoding

        Parameters
        ----------
        tgt : torch.Tensor
            model's target sequence

        Returns
        ----------
        torch.Tensor
            boolean mask Tensor displaying locations of tokens in target to
            mask attention to
        """
        # TODO: Implement next word mask for the target, which masks out all subsequent words
        # HINT: Take a look at torch.triu!
        tgt_len = tgt.size(0)  # Sequence length
        mask = torch.triu(torch.ones(tgt_len, tgt_len, device=tgt.device), diagonal=1).bool()
        return mask

    def encode(self, src: torch.Tensor, src_padding_mask: torch.Tensor) -> torch.Tensor:
        """
        Full encoding pass.

        Parameters
        ----------
        src : torch.Tensor
            model's input sequence (in form of input vocab indices)
        src_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the input sequence

        Returns
        ----------
        torch.Tensor
            output of the full encoder pass
        """
        # TODO: Implement the encode function
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        # print("Encoder input shape:", src_emb.shape)
        return self.encoder(src_emb, src_padding_mask)

    def decode(self,
               tgt: torch.Tensor,
               src_encoding:torch.Tensor,
               src_padding_mask: torch.Tensor,
               tgt_padding_mask: torch.Tensor) -> torch.Tensor:
        """
        Full decoding pass.

        Parameters
        ----------
        tgt : torch.Tensor
            model's target sequence
        src_encoding : torch.Tensor
            output of the encoder
        src_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the "encoded input" (src_encoding)
        tgt_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the input (target sequence)

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        # TODO: Implement the decode function
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        tgt_mask = self.get_tgt_mask(tgt)
        return self.decoder(tgt_emb, src_encoding, src_padding_mask, tgt_mask, tgt_padding_mask)

    def forward(self,
                src: torch.Tensor,
                tgt: torch.Tensor,
                src_padding_mask: torch.Tensor,
                tgt_padding_mask: torch.Tensor) -> torch.Tensor:
        """
        Full forward pass of the transformer.

        Parameters
        ----------
        src : torch.Tensor
            model's input sequence (in form of input vocab indices)
        tgt : torch.Tensor
            model's target sequence (in form of target vocab indices)
        src_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the input sequence
        tgt_padding_mask : torch.Tensor
            boolean mask Tensor displaying locations of padding tokens in
            the target sequence

        Returns
        ----------
        torch.Tensor
            output of the forward pass
        """
        # print("Entering transformer forward pass")
        # TODO: Implement the forward pass
        src_encoding = self.encode(src, src_padding_mask)
        # print("Encoder output shape:", src_encoding.shape)
        # Decode target sequence
        output = self.decode(tgt, src_encoding, src_padding_mask, tgt_padding_mask)
        # print("Decoder output shape:", output.shape)
        # Project to vocabulary
        logits = self.out(output)
        # print("Final logits shape:", logits.shape)

        return logits

### Training the Model



In [37]:
def padding_mask(idx_tensor: torch.Tensor) -> torch.Tensor:
    """
    Returns a boolean mask where True indicates that the token is a padding token.

    Parameters
    ----------
    idx_tensor : torch.Tensor
        tensor for which a padding mask should be generated

    Returns
    ----------
    torch.Tensor
        boolean mask of padding of input tensor
    """
    return (idx_tensor == PAD_IDX).transpose(0, 1)

In [38]:
def train_epoch(model: nn.Module,
                train_dataloader: DataLoader,
                loss_fn: nn.Module,
                optimizer: torch.optim.Optimizer) -> float:
    """
    Trains the inputted model using the provided data, optimizer, and loss
    function for one epoch. Returns the average loss of the epoch.

    Parameters
    ----------
    model : nn.Module
        model to train
    optimizer : torch.optim.Optimizer
        optimizer to use with training
    train_dataloader : DataLoader
        training data
    loss_fn : nn.Module
        loss function to use with training

    Returns
    ----------
    float
        epoch average loss
    """
    model.train()
    total_loss = 0

    for src, tgt in train_dataloader:
        src, tgt = src.to(device), tgt.to(device)
        tgt_input = tgt[:-1, :]

        src_padding_mask = padding_mask(src)
        tgt_padding_mask = padding_mask(tgt_input)

        logits = model(src, tgt_input, src_padding_mask, tgt_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_dataloader)

def val(model: nn.Module, val_dataloader: DataLoader, loss_fn: nn.Module) -> float:
    """
    Performs validation on the inputted model using the provided data and loss
    function for one epoch. Returns the validation loss.

    Parameters
    ----------
    model : nn.Module
        model to train
    val_dataloader : DataLoader
        validation data
    loss_fn : nn.Module
        loss function to use to calculate validation loss

    Returns
    ----------
    float
        average validation loss
    """
    model.eval()
    losses = 0
    for src, tgt in val_dataloader:
        src, tgt = src.to(device), tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_padding_mask = padding_mask(src)
        tgt_padding_mask = padding_mask(tgt_input)

        logits = model(src, tgt_input, src_padding_mask, tgt_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

In [39]:
def train(model: nn.Module,
          train_dataloader : DataLoader,
          loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          n_epochs: int = 10):
    """
    Trains the inputted model using the provided data, optimizer, and loss
    function for n epochs. Prints the average loss of the training epoch
    and the validation loss after each epoch.

    Parameters
    ----------
    model : nn.Module
        model to train
    train_dataloader : DataLoader
        training data
    loss_fn : nn.Module
        loss function to use with training
    optimizer : torch.optim.Optimizer
        optimizer to use with training
    n_epochs : int
        number of epochs to train for
    """

    model = model.to(device)

    for epoch in range(1, n_epochs+1):
        print(f"Epoch {epoch},", end = " ")
        start_time = timer()
        train_loss = train_epoch(model, train_dataloader, loss_fn, optimizer)
        end_time = timer()
        print(f" Train loss: {train_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s", end = " ")

        val_loss = val(model, val_loader, loss_fn)
        print(f" Val loss: {val_loss:.3f}")

In [40]:
# initialize the transformer model
manual_transformer = ManualTransformer(num_encoder_layers=3, num_decoder_layers=3, emb_size=512, nhead=8, src_vocab_size=len(de_vocab), tgt_vocab_size=len(en_vocab))

# define our loss function and optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(manual_transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# train the model, change n_epochs to 15 when you're ready!
train(manual_transformer, train_loader, loss_fn, optimizer, n_epochs=15)

Epoch 1,  Train loss: 4.005, Epoch time = 44.758s  Val loss: 3.058
Epoch 2,  Train loss: 2.804, Epoch time = 44.570s  Val loss: 2.473
Epoch 3,  Train loss: 2.336, Epoch time = 44.842s  Val loss: 2.186
Epoch 4,  Train loss: 2.032, Epoch time = 45.163s  Val loss: 1.986
Epoch 5,  Train loss: 1.815, Epoch time = 44.741s  Val loss: 1.895
Epoch 6,  Train loss: 1.643, Epoch time = 44.273s  Val loss: 1.811
Epoch 7,  Train loss: 1.505, Epoch time = 44.571s  Val loss: 1.773
Epoch 8,  Train loss: 1.390, Epoch time = 42.865s  Val loss: 1.740
Epoch 9,  Train loss: 1.290, Epoch time = 42.721s  Val loss: 1.727
Epoch 10,  Train loss: 1.202, Epoch time = 43.453s  Val loss: 1.715
Epoch 11,  Train loss: 1.123, Epoch time = 42.806s  Val loss: 1.706
Epoch 12,  Train loss: 1.051, Epoch time = 44.083s  Val loss: 1.706
Epoch 13,  Train loss: 0.986, Epoch time = 46.650s  Val loss: 1.724
Epoch 14,  Train loss: 0.926, Epoch time = 47.321s  Val loss: 1.744
Epoch 15,  Train loss: 0.872, Epoch time = 44.844s  Val l

Congrats! You've finished Part 1.

# Part 2: Evaluating our Transformer



## Getting Translations Out of Our Model
Before we can actually calculate BLEU though, we need to get our outputs from the model.

### Greedy Decode vs. Beam Search 


In [46]:
from torch.nn.functional import softmax
import heapq

def greedy_decode(model: nn.Module,
                  src: torch.Tensor,
                  max_len: int,
                  start_symbol: int
                  ) -> tuple[torch.Tensor, float]:
    """
    Function to generate a translation using a greedy decoding algorithm

    Parameters
    ----------
    model : nn.Module
        trained model to decode from
    src : torch.Tensor
        sequence inputted into model
    max_len : int
        maximum length to generate with decoding
    optimizer : torch.optim.Optimizer
        optimizer to use with training
    start_symbol : int
        vocabulary index to the start of sentence token

    Returns
    ----------
    tuple[torch.Tensor, float]
        a tuple of a tensor containing the sentence found using greedy
        decode represented using vocabulary indices, and the probability
        associated with that sentence
    """

    src = src.to(device)
    src_padding_mask = padding_mask(src)

    total_prob = 1.0

    src_encoding = model.encode(src, src_padding_mask).to(device)
    seq = torch.ones((1, 1), dtype=torch.long, device=device).fill_(start_symbol)
    for i in range(max_len-1):
        tgt_padding_mask = padding_mask(seq)
        out = model.decode(seq, src_encoding, src_padding_mask, tgt_padding_mask).to(device)
        out = out.transpose(0, 1).to(device)
        probs = softmax(model.out(out[:, -1]), dim=-1)
        prob, next_word = torch.max(probs, dim=1)
        total_prob *= prob.item()
        next_word = next_word.item()

        seq = torch.cat([seq, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)

        if next_word == EOS_IDX:
            break
    return seq.flatten(), total_prob

def beam_search(model: nn.Module,
                src: torch.Tensor,
                max_len: int,
                start_symbol: int,
                beam_size: int
                ) -> tuple[torch.Tensor, float]:
    """
    Function to generate a translation using a beam search algorithm

    Parameters
    ----------
    model : nn.Module
        trained model to decode from
    src : torch.Tensor
        sequence inputted into model
    max_len : int
        maximum length to generate with decoding
    optimizer : torch.optim.Optimizer
        optimizer to use with training
    start_symbol : int
        vocabulary index to the start of sentence token
    beam_size : int
        size of the beam using during the search

    Returns
    ----------
    tuple[torch.Tensor, float]
        a tuple of a tensor containing the sentence found using beam
        search represented using vocabulary indices, and the probability
        associated with that sentence
    """
    # TODO: Implement beam search
    src = src.to(device)
    src_padding_mask = padding_mask(src)

    src_encoding = model.encode(src, src_padding_mask).to(device)

    # Priority queue for beam candidates (negative log prob to use heapq as min-heap)
    beam = [([start_symbol], 0.0)]
    completed_sequences = []

    for _ in range(max_len):
        new_beam = []

        for seq, score in beam:
            if seq[-1] == EOS_IDX:
                # If EOS is reached, add the sequence to completed sequences
                completed_sequences.append((seq, score))
                continue

            # Prepare the input sequence for decoding
            seq_tensor = torch.tensor(seq, dtype=torch.long, device=device).unsqueeze(1)
            tgt_padding_mask = padding_mask(seq_tensor)

            # Get model predictions
            out = model.decode(seq_tensor, src_encoding, src_padding_mask, tgt_padding_mask).to(device)
            out = out.transpose(0, 1).to(device)
            probs = softmax(model.out(out[:, -1]), dim=-1)

            # Get top beam_size predictions
            top_probs, top_indices = torch.topk(probs, beam_size)

            # Add new hypotheses to the new beam
            for i in range(beam_size):
                word = top_indices[0, i].item()
                prob = top_probs[0, i].item()
                new_seq = seq + [word]
                new_score = score - torch.log(torch.tensor(prob)).item()  # Add negative log prob
                new_beam.append((new_seq, new_score))

        # Keep the top beam_size sequences
        beam = heapq.nsmallest(beam_size, new_beam, key=lambda x: x[1])  # Min-heap by score

        # If all sequences are completed, break early
        if len(completed_sequences) >= beam_size:
            break

    # Choose the best completed sequence
    if completed_sequences:
        best_seq, best_score = min(completed_sequences, key=lambda x: x[1])
    else:
        best_seq, best_score = min(beam, key=lambda x: x[1])

    return torch.tensor(best_seq, dtype=torch.long, device=device), torch.exp(torch.tensor(-best_score))



In [49]:
def tokenlist_to_strlist(tokens: torch.Tensor, vocab: torchtext.vocab.Vocab) -> list[str]:
    """
    Converts a tensor representation of a list of tokens to a list of strings.

    Parameters
    ----------
    tokens : torch.Tensor
        tensor representation of a list of tokens
    vocab : torchtext.vocab.Vocab
        vocabulary associated with the given tokens

    Returns
    ----------
    list[str]
        returns a representation of the tokens in the form of a list of word strings
    """
    return vocab.lookup_tokens(list(tokens.cpu().numpy()))

def tokenlist_to_str(tokens: torch.Tensor,
                     vocab: torchtext.vocab.Vocab,
                     show_pad: bool = False
                     ) -> str:
    """
    Converts a tensor representation of a list of tokens to a string, ignoring start
    and end of sentence tokens, and optionally ignoring padding tokens.

    Parameters
    ----------
    tokens : torch.Tensor
        tensor representation of a list of tokens
    vocab : torchtext.vocab.Vocab
        vocabulary associated with the given tokens
    show_pad : bool
        whether the outputted string should contain pad tokens or not

    Returns
    ----------
    str
        returns a representation of the tokens in the form of a string
    """
    out = " ".join(tokenlist_to_strlist(tokens, vocab)).replace("<bos>", "").replace("<eos>", "")
    if show_pad:
        return out
    else:
        return out.replace("<pad>", "")

def translate_tokens(model: nn.Module, src_tokens: torch.Tensor) -> tuple[torch.Tensor, float]:
    """
    Takes a tensor of source tokens and "translates" them by inputting them into
    the model and using either greedy decode or beam search to decode the output

    Parameters
    ----------
    model : nn.Module
        trained model to input tokens into
    src_tokens : torch.Tensor
        tensor representations of input sentences without start and end of
        sentence indices

    Returns
    ----------
    tuple[torch.Tensor, float]
        a tuple of a tensor containing the sentence found using either greedy
        decode or beam search represented using vocabulary indices, and the
        probability associated with that sentence
    """
    model.eval()
    src_tokens = torch.cat([torch.tensor([BOS_IDX]), src_tokens, torch.tensor([EOS_IDX])], dim=0).view(-1, 1)
    num_tokens = src_tokens.shape[0]

    # TODO: uncomment the below line and comment out the greedy_decode line to run beam_search!
    # tgt_tokens, prob = beam_search(model, src_tokens, num_tokens + 5, BOS_IDX, 5)
    tgt_tokens, prob = greedy_decode(model, src_tokens, num_tokens + 5, BOS_IDX)
    return tgt_tokens, prob

def translate(model: nn.Module,
              src_sentence: str,
              output_tokens: bool=False
              ) -> tuple[str, torch.Tensor, torch.Tensor] | tuple[str, float]:
    """
    Takes a sentence string and "translates" it

    Parameters
    ----------
    model : nn.Module
        trained model to input tokens into
    src_sentence : str
        string representation of input sentence
    output_tokens : bool
        whether the source and target tokens are also outputted

    Returns
    ----------
    tuple[str, torch.Tensor, torch.Tensor]
        a string representation of the translated sentence, the tensor
        representation of the input, and the tensor representation of the
        translated sentence. Used in analyze_sentence below
    OR
    tuple[str, float]
        a string represention of the translated sentence and the
        probability associated with that sentence
    """
    model.eval()
    src_tokens = torch.tensor(de_vocab(de_tokenizer(src_sentence)), dtype=torch.long)
    tgt_tokens, prob = translate_tokens(model, src_tokens)
    translation = tokenlist_to_str(tgt_tokens, en_vocab)
    if output_tokens:
      return translation, src_tokens, tgt_tokens
    else:
      return translation, prob

### Take a look at the outputs!



In [50]:
def pretty_print(translation: str, prob: float):
    """
    Prints a translation and its associated probability nicely

    Parameters
    ----------
    translation : str
        translation string
    prob : float
        probability associated with translation
    """
    print(translation, f"Probability: {prob}")
    print()

pretty_print(*translate(manual_transformer, "Ein Mann auf einem Maisfeld mit einem Hund"))
pretty_print(*translate(manual_transformer, "Drei Kinder essen Eis, während ein Elternteil zuschaut"))
pretty_print(*translate(manual_transformer, "Eine Frau am Strand fotografiert das Meer"))
pretty_print(*translate(manual_transformer, "Ein Traktor bewegt Erde für den Bau einer Stützmauer."))

 A man on a ice field with a dog . 
  Probability: 0.0009770050634459462

 Three children eating ice as a parent watches . 
  Probability: 0.05722860927290408

 A woman on the beach is taking pictures of the ocean . 
 Probability: 0.04068607463024233

 A tractor moving dirt for the wall for construction 
  Probability: 0.0002418453983292781



## Evaluating Translations using BLEU

### Implement BLEU


In [51]:
import math

def my_corpus_bleu(list_reference: list, list_candidate: list, n: int=4):
    """
    Calculates the BLEU score given a list of references and a list of candidates

    Parameters
    ----------
    list_reference : list[list[str]] | list[list[int]]
      list of reference sentences each in the form of a list of words or vocab indices
    list_candidate : list[list[str]] | list[list[int]]
      list of candidate sentences each in the form of a list of words or vocab indices

    Returns
    -------
    float
      the BLEU score given the list of references and candidates
    """
    # TODO: Implement the BLEU algorithm
    assert len(list_reference) == len(list_candidate), "Mismatched reference and candidate sizes."

    precisions = [0] * n
    total_n_grams = [0] * n
    ref_length = 0
    cand_length = 0

    for ref, cand in zip(list_reference, list_candidate):
        ref_length += len(ref)
        cand_length += len(cand)

        for i in range(1, n + 1):
            cand_ngrams = Counter(tuple(cand[j:j + i]) for j in range(len(cand) - i + 1))
            ref_ngrams = Counter(tuple(ref[j:j + i]) for j in range(len(ref) - i + 1))


            for ngram, cand_count in cand_ngrams.items():
                precisions[i - 1] += min(cand_count, ref_ngrams.get(ngram, 0))
            total_n_grams[i - 1] += sum(cand_ngrams.values())


    ngram_precisions = [
        (precisions[i] / total_n_grams[i]) if total_n_grams[i] > 0 else 0
        for i in range(n)
    ]

    # Compute brevity penalty
    if cand_length > ref_length:
        brevity_penalty = 1.0
    else:
        brevity_penalty = math.exp(1 - ref_length / cand_length) if cand_length > 0 else 0

    # Compute BLEU score
    bleu = brevity_penalty * math.exp(sum(math.log(p) for p in ngram_precisions if p > 0) / n)

    return bleu

### Checking our BLEU implementation


In [52]:
from nltk.translate.bleu_score import corpus_bleu

toy_references = ["The NASA Opportunity rover is battling a massive dust storm on Mars .".split()]
toy_hypothesis = ["A NASA rover is fighting a massive storm on Mars .".split()]

toy_references2 = ["The quick brown fox jumped over the lazy dog".split(),
                   "holy crap today is fring friday".split(),
                   "jesse we have to cook". split()]
toy_hypothesis2 = ["The fast maroon fox jumped over the tired old dog".split(),
                   "oh wow today is fring friday".split(),
                   "jesse we need to cook".split()]


print("Student:", my_corpus_bleu(toy_references, toy_hypothesis, 4))
print("NLTK:", corpus_bleu([[lst] for lst in toy_references], toy_hypothesis)) # labels must be a list of list of words for NLTK BLEU
print()
print("Student:", my_corpus_bleu(toy_references2, toy_hypothesis2, 4))
print("NLTK:", corpus_bleu([[lst] for lst in toy_references2], toy_hypothesis2)) # labels must be a list of list of words for NLTK BLEU

Student: 0.2722179122549562
NLTK: 0.27221791225495623

Student: 0.3387551654364099
NLTK: 0.33875516543640977


In [53]:
from nltk.translate.bleu_score import corpus_bleu

# remember manual_transformer, val_data, and translate_tokens?
predictions = [translate_tokens(manual_transformer, t[0])[0].tolist()[1:-1] for t in val_data]
student_labels = [t[1].tolist() for t in val_data] # labels is a list of words for student BLEU
nltk_labels = [[t[1].tolist()] for t in val_data] # labels must be a list of list of words for NLTK BLEU

print("Student BLEU:", my_corpus_bleu(student_labels, predictions))
print("NLTK BLEU:", corpus_bleu(nltk_labels, predictions))

Student BLEU: 0.3902789602093695
NLTK BLEU: 0.3902789602093695
